In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
### Testing the Fashion MNIST Dataset

In [6]:
### Downloading training data from open datasets.

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

26422272it [00:26, 1010474.32it/s]                                                                                                                                          


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



29696it [00:00, 187148.76it/s]                                                                                                                                              


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



4422656it [00:02, 1945842.10it/s]                                                                                                                                           


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



6144it [00:00, 2147304.71it/s]                                                                                                                                              

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [9]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [18]:
batch_size = 64

# Creating the dataloaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
tests_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in tests_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

In [20]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

using cuda device


In [21]:
### Defining a model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the model params

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
"""
In a single training step:
    The model makes prediction on the training datasets (fed in batches)
    And back-propagates the predicition errors to adjust the model params
    
"""

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Calculate the prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")


    Writing a function to check the performance of the model 
    against the test dataset


In [30]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0 
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [ ]:
The training process is conducted over various epochs or iterations. 
During each epoch the model learns parameters to make better predictions.
We print the models accuracy/loss at each epoch.

Ideally, we want that the accuracy increases and the loss decreases with each epoch.

In [31]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(tests_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------
loss : 1.564231 [    0/60000]
loss : 1.528558 [ 6400/60000]
loss : 1.387985 [12800/60000]
loss : 1.467457 [19200/60000]
loss : 1.336430 [25600/60000]
loss : 1.334263 [32000/60000]
loss : 1.345402 [38400/60000]
loss : 1.283904 [44800/60000]
loss : 1.310115 [51200/60000]
loss : 1.216826 [57600/60000]
Test Error: 
 Accuracy: 63.1%, Avg loss: 1.244187 

Epoch 2
-----------------------
loss : 1.317175 [    0/60000]
loss : 1.298281 [ 6400/60000]
loss : 1.141680 [12800/60000]
loss : 1.251946 [19200/60000]
loss : 1.119384 [25600/60000]
loss : 1.139459 [32000/60000]
loss : 1.154891 [38400/60000]
loss : 1.110339 [44800/60000]
loss : 1.138243 [51200/60000]
loss : 1.060790 [57600/60000]
Test Error: 
 Accuracy: 64.7%, Avg loss: 1.082703 

Epoch 3
-----------------------
loss : 1.151347 [    0/60000]
loss : 1.151645 [ 6400/60000]
loss : 0.977579 [12800/60000]
loss : 1.116993 [19200/60000]
loss : 0.985756 [25600/60000]
loss : 1.007058 [32000/60000]
loss : 1.038781 [384

In [33]:
### Saving Models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [34]:
### Loading models

In [37]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
### Inference

In [38]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [39]:
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [41]:
x, y = test_data[0][0], test_data[0][1]

In [43]:
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted} , Actual: {actual}")

Predicted: Ankle boot , Actual: Ankle boot
